In [1]:
import numpy as np
import json
from keras.utils.np_utils import to_categorical
import time
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [4]:
with open('Random_Smart_Losses.json') as json_file:
    training = json.load(json_file)

In [5]:
for item in [training]:
    print(len(item['moves']))
    print(len(item['boards']))

54620
54620


In [6]:
X = np.array(training['boards'])
y = to_categorical(training['moves'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.2, random_state=42)

In [7]:
for i in [X_train, X_test, X_val, y_train, y_test, y_val]:
    print(len(i))

49158
4369
1093
49158
4369
1093


In [9]:
from keras.models import load_model
parial_model = load_model('smart_model_partial.h5')

In [12]:
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics


history = parial_model.fit(X_train,
                   y_train,
                   epochs=10,
                   batch_size = 512,
                   validation_data = (X_test, y_test))

Train on 49158 samples, validate on 4369 samples
Epoch 1/30
49158/49158 [==============================] - 1s 12us/step - loss: 1.2273 - acc: 0.4859 - val_loss: 1.2353 - val_acc: 0.4875
Epoch 2/30
49158/49158 [==============================] - 1s 11us/step - loss: 1.2267 - acc: 0.4849 - val_loss: 1.2329 - val_acc: 0.4878
Epoch 3/30
49158/49158 [==============================] - 1s 11us/step - loss: 1.2269 - acc: 0.4850 - val_loss: 1.2325 - val_acc: 0.4891
Epoch 4/30
49158/49158 [==============================] - 1s 11us/step - loss: 1.2270 - acc: 0.4859 - val_loss: 1.2352 - val_acc: 0.4804
Epoch 5/30
49158/49158 [==============================] - 1s 12us/step - loss: 1.2269 - acc: 0.4857 - val_loss: 1.2364 - val_acc: 0.4765
Epoch 6/30
49158/49158 [==============================] - 1s 14us/step - loss: 1.2273 - acc: 0.4862 - val_loss: 1.2340 - val_acc: 0.4797
Epoch 7/30
49158/49158 [==============================] - 1s 13us/step - loss: 1.2287 - acc: 0.4860 - val_loss: 1.2377 - val_acc:

In [13]:
parial_model.save('loss_model.h5')

In [11]:
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics

new_model = models.Sequential()

new_model.add(layers.Dense(729, activation = 'relu', input_shape = (9,)))
new_model.add(layers.Dense(729, activation = 'relu'))
new_model.add(layers.Dense(729, activation = 'relu'))
new_model.add(layers.Dense(81, activation = 'relu'))
new_model.add(layers.Dense(81, activation = 'relu'))
new_model.add(layers.Dense(81, activation = 'relu'))
new_model.add(layers.Dense(9, activation = 'softmax'))



new_model.compile(optimizer = 'rmsprop',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])


# train the network
now = time.time()

history = new_model.fit(X_train,
                   y_train,
                   epochs=10,
                   batch_size = 512,
                   validation_data = (X_test, y_test))

print(time.time()-now)

print(new_model.evaluate(X_val, y_val))

Train on 49158 samples, validate on 4369 samples
Epoch 1/10
49158/49158 [==============================] - 1s 18us/step - loss: 1.4897 - acc: 0.4216 - val_loss: 1.5075 - val_acc: 0.4317
Epoch 2/10
49158/49158 [==============================] - 1s 11us/step - loss: 1.2771 - acc: 0.4758 - val_loss: 1.4216 - val_acc: 0.4278
Epoch 3/10
49158/49158 [==============================] - 1s 11us/step - loss: 1.2550 - acc: 0.4789 - val_loss: 1.3030 - val_acc: 0.4836
Epoch 4/10
49158/49158 [==============================] - 1s 11us/step - loss: 1.2464 - acc: 0.4808 - val_loss: 1.3100 - val_acc: 0.4704
Epoch 5/10
49158/49158 [==============================] - 1s 11us/step - loss: 1.2430 - acc: 0.4792 - val_loss: 1.2970 - val_acc: 0.4669
Epoch 6/10
49158/49158 [==============================] - 1s 12us/step - loss: 1.2391 - acc: 0.4817 - val_loss: 1.2741 - val_acc: 0.4743
Epoch 7/10
49158/49158 [==============================] - 1s 11us/step - loss: 1.2385 - acc: 0.4817 - val_loss: 1.3218 - val_acc:

In [55]:
model.predict(np.array([[0,0,0,0,0,0,0,0,0]]))

array([[0.02178787, 0.02496707, 0.0257297 , 0.02728997, 0.78476787,
        0.03005825, 0.0259759 , 0.02616759, 0.03325576]], dtype=float32)

In [56]:
model.predict(np.array([[1,-1,0
                         ,1,0,0
                         ,0,-1,0]]))

array([[0.1683737 , 0.05102368, 0.08542217, 0.04528546, 0.22401215,
        0.06745588, 0.0763906 , 0.13408835, 0.14794804]], dtype=float32)

In [45]:
test_model.predict(np.array([[1,-1,0
                         ,1,0,0
                         ,0,-1,0]]))

array([[0.00043794, 0.00052062, 0.21343212, 0.00179795, 0.2019275 ,
        0.16881675, 0.31666756, 0.00581618, 0.09058337]], dtype=float32)

In [47]:
model.predict(np.array([[1,-1,0
                         ,0,1,0
                         ,0,-1,0]]))

array([[0.00325159, 0.00093395, 0.19345361, 0.16106528, 0.00418115,
        0.12907228, 0.23651929, 0.00877786, 0.26274502]], dtype=float32)

In [48]:
test_model.predict(np.array([[1,-1,0
                         ,0,1,0
                         ,0,-1,0]]))

array([[1.1218474e-05, 3.7624336e-06, 3.2174754e-01, 3.4098116e-01,
        5.0292820e-05, 2.4599876e-01, 3.2355905e-02, 1.3134971e-04,
        5.8719952e-02]], dtype=float32)

In [50]:
model.predict(np.array([[1,0,0
                         ,0,1,0
                         ,0,0,0]]))

array([[2.5778514e-04, 2.6437433e-02, 1.2027474e-01, 5.6764647e-02,
        5.4575043e-04, 9.6586965e-02, 2.5654316e-01, 1.6845371e-01,
        2.7413586e-01]], dtype=float32)

In [51]:
test_model.predict(np.array([[1,0,0
                         ,0,1,0
                         ,0,0,0]]))

array([[2.26156026e-05, 3.40315397e-03, 2.33651400e-02, 2.84878999e-01,
        9.09274968e-05, 2.25693494e-01, 5.71935624e-03, 1.10313356e-01,
        3.46512944e-01]], dtype=float32)

In [ ]:
### the above move prediction isn't bad, but it blocks instead of going for the win

In [35]:
from keras.models import load_model
test_model = load_model('smart_model.h5')

In [36]:
test_model.predict(np.array([[0,0,0,0,0,0,0,0,0]]))

array([[0.02982613, 0.02642992, 0.04474984, 0.04018626, 0.70327   ,
        0.04147457, 0.03849569, 0.03270375, 0.04286384]], dtype=float32)

In [37]:
model.predict(np.array([[1,0,0
                         ,1,0,0
                         ,0,0,0]]))

array([[0.00406162, 0.01751591, 0.04555007, 0.00840351, 0.17237176,
        0.21559587, 0.06073135, 0.2012618 , 0.27450812]], dtype=float32)

In [ ]:
model.predict(np.array([[1,0,0
                         ,0,1,0
                         ,0,0,0]]))

In [ ]:
current_board = [[0,0,0],[0,0,0],[0,0,0]]

def board_to_prediction_list(board):
    vector_board = []
    for row in board:
        vector_board += row
    np_board = np.array([vector_board])

    return list(model.predict(np_board)[0])

def make_prediction(predicted_moves, available_moves):
    
    while True:
        move = predicted_moves.index(max(predicted_moves))
        predicted_moves[move] = 0

        if move in available_moves:
            return move

pred_list = board_to_prediction_list(current_board)
print(pred_list)
my_move = make_prediction(pred_list,set([3,5,6,7,8]))
print(my_move)

In [ ]:
predicted_moves.index(max(predicted_moves))

In [58]:
# this is the model I used
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics

model = models.Sequential()

model.add(layers.Dense(729, activation = 'relu', input_shape = (9,)))
model.add(layers.Dense(729, activation = 'relu'))
model.add(layers.Dense(729, activation = 'relu'))
model.add(layers.Dense(81, activation = 'relu'))
model.add(layers.Dense(81, activation = 'relu'))
model.add(layers.Dense(81, activation = 'relu'))
model.add(layers.Dense(9, activation = 'softmax'))



model.compile(optimizer = 'rmsprop',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])


# train the network
now = time.time()

history = model.fit(X_train,
                   y_train,
                   epochs=10,
                   batch_size = 512,
                   validation_data = (X_test, y_test))

print(time.time()-now)

print(model.evaluate(X_val, y_val))

Train on 270276 samples, validate on 54056 samples
Epoch 1/10
270276/270276 [==============================] - 4s 14us/step - loss: 0.7634 - acc: 0.7829 - val_loss: 0.6116 - val_acc: 0.8221
Epoch 2/10
270276/270276 [==============================] - 3s 12us/step - loss: 0.6117 - acc: 0.8220 - val_loss: 0.6114 - val_acc: 0.8216
Epoch 3/10
270276/270276 [==============================] - 3s 12us/step - loss: 0.5977 - acc: 0.8235 - val_loss: 0.6125 - val_acc: 0.8229
Epoch 4/10
270276/270276 [==============================] - 3s 12us/step - loss: 0.5911 - acc: 0.8245 - val_loss: 0.6060 - val_acc: 0.8199
Epoch 5/10
270276/270276 [==============================] - 4s 14us/step - loss: 0.5870 - acc: 0.8252 - val_loss: 0.5926 - val_acc: 0.8232
Epoch 6/10
270276/270276 [==============================] - 3s 12us/step - loss: 0.5846 - acc: 0.8258 - val_loss: 0.5970 - val_acc: 0.8223
Epoch 7/10
270276/270276 [==============================] - 3s 12us/step - loss: 0.5818 - acc: 0.8260 - val_loss: 0

In [59]:
model.save('smart_model.h5')